# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298475914216                   -0.85    5.0         
  2   -8.300192020816       -2.77       -1.25    1.0   93.5ms
  3   -8.300428808838       -3.63       -1.89    2.1    111ms
  4   -8.300460033584       -4.51       -2.76    2.9    155ms
  5   -8.300463786534       -5.43       -3.02    2.6    130ms
  6   -8.300464190097       -6.39       -3.19    6.9    167ms
  7   -8.300464408468       -6.66       -3.33    2.8    129ms
  8   -8.300464532862       -6.91       -3.47    1.5    119ms
  9   -8.300464618330       -7.07       -3.70    1.4    123ms
 10   -8.300464623249       -8.31       -3.75    1.1    124ms
 11   -8.300464638076       -7.83       -3.95    1.8    116ms
 12   -8.300464643767       -8.24      

In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67448358029                   -0.70    6.4         
  2   -16.67862634052       -2.38       -1.14    1.0    255ms
  3   -16.67920999193       -3.23       -1.87    3.0    298ms
  4   -16.67927919484       -4.16       -2.76    3.4    321ms
  5   -16.67928597831       -5.17       -3.19    5.1    394ms
  6   -16.67928619560       -6.66       -3.48    2.4    288ms
  7   -16.67928621810       -7.65       -3.93    2.2    269ms
  8   -16.67928622144       -8.48       -4.57    2.9    296ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32440383643                   -0.56    6.6         
  2   -33.33243280630       -2.10       -1.00    1.1    933ms
  3   -33.33406297969       -2.79       -1.74    5.4    1.29s
  4   -33.33677280739       -2.57       -2.39    6.4    1.38s
  5   -33.33694008617       -3.78       -2.75   10.2    1.56s
  6   -33.33694348292       -5.47       -3.58    2.4    977ms
  7   -33.33694373952       -6.59       -3.94    5.4    1.45s
  8   -33.33694378118       -7.38       -4.63    3.0    1.09s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298405501050                   -0.85    5.1         
  2   -8.300265954758       -2.73       -1.59    1.1   80.0ms
  3   -8.300439816914       -3.76       -2.63    3.2    115ms
  4   -8.300417244283   +   -4.65       -2.42    6.1    183ms
  5   -8.300463714557       -4.33       -3.29    1.0   84.2ms
  6   -8.300464594357       -6.06       -3.76    2.5    121ms
  7   -8.300464639718       -7.34       -4.42    2.1    101ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31458101752                   -0.56    6.6         
  2   -33.27295606451   +   -1.38       -1.25    1.6    826ms
  3   +10.11543627306   +    1.64       -0.25    6.8    1.92s
  4   -33.32376597635        1.64       -1.67    6.1    1.82s
  5   -33.05183090343   +   -0.57       -1.24    4.5    1.44s
  6   -31.78240147965   +    0.10       -0.96    4.8    1.40s
  7   -33.31211919250        0.18       -1.79    4.2    1.32s
  8   -33.31743536104       -2.27       -1.68    2.2    1.05s
  9   -33.33214010409       -1.83       -2.00    2.5    1.07s
 10   -33.33673291564       -2.34       -2.66    2.1    966ms
 11   -33.33690584308       -3.76       -2.92    3.4    1.25s
 12   -33.33692223616       -4.79      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.